<a href="https://colab.research.google.com/github/simplifiedcomputing/weed_detection/blob/main/weed_detection.ipynb" target="_blank"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Weed Detection
Locate medical plants for human and animal health using object detection, segmentation and classification.

## Install

- FastAI (training loop library)
- IceVision (computer vision framework) -> [forum](https://discord.gg/JDBeZYK)
- MMDetection and Yolo v5 (neural net models)

In [ ]:
!pip install openmim -q
!mim install mmcv-full
!mim install mmdet

!pip install git+git://github.com/airctic/icevision.git#egg=icevision[all] -U -q
!pip install git+git://github.com/airctic/icedata.git -U -q
!pip install yolov5-icevision -U -q 

In [ ]:
# Restart kernel after installation
exit()

## Imports
Import the required components.


In [ ]:
from icevision.all import *

## Collect Data

Drone parameters for DJI MAVIC 2 Zoom

- Autopilot (in planning)
- Hoover and Capture to minimize image distortion
- 5m flight altitude
- 5,5m capture distance (image size 7,1m x 5,5m at 5m altitude)  
- bright and windless weather conditions

## Prepare the Dataset

### Labeling for Drone collected data
Labeling in PascalVOC format

https://github.com/tzutalin/labelImg or https://github.com/openvinotoolkit/cvat

### Parse the dataset

The parser loads the annotation file and parses them returning a list of training and validation records. The parsers support multiple formats (including VOC and COCO).


In [ ]:
# Set local data dir path
data_dir = Path('/home/martin/projekte/herbst/data')

In [3]:
# or download dataset
url = "https://simplified-computing.de/herbst21.zip"
dest_dir = "herbst"
data_dir = icedata.load_data(url, dest_dir)

NameError: name 'icedata' is not defined

In [ ]:
# Create the parser
parser = parsers.VOCBBoxParser(annotations_dir=data_dir / "annotations_herbst21_training", images_dir=data_dir / "image_herbst21_training")

#### Sample dataset "fridge"

In [ ]:
# Download dataset fridge
#url = "https://cvbp-secondary.z19.web.core.windows.net/datasets/object_detection/odFridgeObjects.zip"
#dest_dir = "fridge"
#data_dir_fridge = icedata.load_data(url, dest_dir)

In [ ]:
# Create the parser for fridge
#parser = parsers.VOCBBoxParser(annotations_dir=data_dir_fridge / "odFridgeObjects/annotations", images_dir=data_dir_fridge / "odFridgeObjects/images")

In [ ]:
# Parse annotations to create records
train_records, valid_records = parser.parse()
parser.class_map

### Explore Date

Using IceVision Dashboard

https://pypi.org/project/icevision-dashboards/

### Creating datasets with agumentations and transforms

To the training set we apply the Albumentation's default `aug_tfms`. It randomly applies broadly useful transformations including rotation, cropping, horizintal flips, and more. 

The validation set is only resized (with padding).

In [1]:
# Transforms
# size is set to 384 because EfficientDet requires its inputs to be divisible by 128
image_size = 2048
train_tfms = tfms.A.Adapter([*tfms.A.aug_tfms(size=image_size, presize=512), tfms.A.Normalize()])
valid_tfms = tfms.A.Adapter([*tfms.A.resize_and_pad(image_size), tfms.A.Normalize()])

NameError: name 'tfms' is not defined

We then create `Datasets` for both. The dataset applies the transforms to the annotations (bounding boxes) and images in the data records.

In [ ]:
# Datasets
train_ds = Dataset(train_records, train_tfms)
valid_ds = Dataset(valid_records, valid_tfms)

#### Understanding the transforms

The Dataset transforms are only applied when we grab (get) an item. Several of the default `aug_tfms` have a random element to them. For example, one might perform a rotation with probability 0.5 where the angle of rotation  is randomly selected between +45 and -45 degrees.

This means that the learner sees a slightly different version of an image each time it is accessed. This effectively increases the size of the dataset and improves learning.

We can look at result of getting the 0th image from the dataset a few times and see the differences. Each time you run the next cell, you will see different results due to the random element in applying transformations.

In [ ]:
# Show an element of the train_ds with augmentation transformations applied
samples = [train_ds[0] for _ in range(3)]
show_samples(samples, ncols=3)

## Select a library, model, and backbone

In order to create a model, we need to:
* Choose one of the **libraries** supported by IceVision
* Choose one of the **models** supported by the library
* Choose one of the **backbones** corresponding to a chosen model

You can access any supported models by following the IceVision unified API, use code completion to explore the available models for each library.



## Creating a model
Selections only take two simple lines of code. For example, to try the mmdet library using the retinanet model and the resnet50_fpn_1x backbone  could be specified by:
```
model_type = models.mmdet.retinanet
backbone = model_type.backbones.resnet50_fpn_1x(pretrained=True)
```
As pretrained models are used by default, we typically leave this out of the backbone creation step.

You can easily pick which option you want to try by setting the value of `selection`. This shows you how easy it is to try new libraries, models, and backbones.


In [ ]:
# Just change the value of selection to try another model

selection = 4

extra_args = {}

if selection == 0:
  model_type = models.mmdet.vfnet
  backbone = model_type.backbones.resnet50_fpn_mstrain_2x

elif selection == 1:
  model_type = models.mmdet.retinanet
  backbone = model_type.backbones.resnet50_fpn_1x
  # extra_args['cfg_options'] = { 
  #   'model.bbox_head.loss_bbox.loss_weight': 2,
  #   'model.bbox_head.loss_cls.loss_weight': 0.8,
  #    }
  
elif selection == 2:
  # The Retinanet model is also implemented in the torchvision library
  model_type = models.torchvision.retinanet
  backbone = model_type.backbones.resnet50_fpn

elif selection == 3:
  model_type = models.ross.efficientdet
  backbone = model_type.backbones.tf_lite0
  # The efficientdet model requires an img_size parameter
  extra_args['img_size'] = image_size

elif selection == 4:
  model_type = models.ultralytics.yolov5
  backbone = model_type.backbones.small
  # The yolov5 model requires an img_size parameter
  extra_args['img_size'] = image_size

model_type, backbone, extra_args

In [ ]:
backbone.__dict__

Now it is just a one-liner to instantiate the model. If you want to try another *option*, just edit the line at the top of the previous cell.

In [ ]:
# Instantiate the model
model = model_type.model(backbone=backbone(pretrained=True), num_classes=len(parser.class_map), **extra_args) 

## Data Loader

The Data Loader is specific to a model_type. The job of the data loader is to get items from a dataset and batch them up in the specific format required by each model. This is why creating the data loaders is separated from creating the datasets.

We can take a look at the first batch of items from the `valid_dl`. Remember that the `valid_tfms` only resized (with padding) and normalized records, so different images, for example, are not returned each time. This is important to provide consistent validation during training.

In [ ]:
# Data Loaders
train_dl = model_type.train_dl(train_ds, batch_size=1, num_workers=1, shuffle=True)
valid_dl = model_type.valid_dl(valid_ds, batch_size=1, num_workers=1, shuffle=False)

In [ ]:
# show batch
model_type.show_batch(first(valid_dl), ncols=4)

## Metrics

The fastai engine collect metrics to track progress during training. IceVision provides metric classes that work across the engines and libraries.

In [ ]:
metrics = [COCOMetric(metric_type=COCOMetricType.bbox)]

## Training

IceVision is an easy to use computer vision framework plugged into DL learning engines such as [fastai2](https://github.com/fastai/fastai2).  


### Training using fastai

In [ ]:
learn = model_type.fastai.learner(dls=[train_dl, valid_dl], model=model, metrics=metrics)

In [ ]:
learn.lr_find()

# For Sparse-RCNN, use lower `end_lr`
# learn.lr_find(end_lr=0.005)

In [ ]:
learn.fine_tune(20, 0.047, freeze_epochs=1)

## Using the model - inference and showing results

The first step in reviewing the model is to show results from the validation dataset. This is easy to do with the `show_results` function.

In [ ]:
model_type.show_results(model, valid_ds, detection_threshold=.5)

### Prediction

Sometimes you want to have more control than `show_results` provides. You can construct an inference dataloader using `infer_dl` from any IceVision dataset and pass this to `predict_dl` and use `show_preds` to look at the predictions.

A prediction is returned as a dict with keys: `scores`, `labels`, `bboxes`, and possibly `masks`. 

Prediction functions that take a `detection_threshold` argument will only return the predictions whose score is above the threshold.

Prediction functions that take a `keep_images` argument will only return the (tensor representation of the) image when it is `True`. In interactive environments, such as a notebook, it is helpful to see the image with bounding boxes and labels applied. In a deployment context, however, it is typically more useful (and efficient) to return the bounding boxes by themselves.



> NOTE: For a more detailed look at inference check out the [inference tutorial](https://airctic.com/dev/inference/)

In [ ]:
infer_dl = model_type.infer_dl(valid_ds, batch_size=4, shuffle=False)
preds = model_type.predict_from_dl(model, infer_dl, keep_images=True)

In [ ]:
show_preds(preds=preds[:4])

## Localization

Calculate GPS coordinates for detected objects.

In [ ]:
%pip install exif -q
from exif import Image

In [ ]:
# load image
img_path = '/home/martin/projekte/herbst/data/image_herbst21_training/DJI_0252.JPG'
img = Image(img_path)
img

In [ ]:
# gps coordinates

# convert to decimal coordinates
def decimal_coords(coords, ref):
 decimal_degrees = coords[0] + coords[1] / 60 + coords[2] / 3600
 if ref == "S" or ref == "W":
     decimal_degrees = -decimal_degrees
 return decimal_degrees

decimal_coords(img.gps_longitude, img.gps_longitude_ref), decimal_coords(img.gps_latitude, img.gps_latitude_ref)

In [ ]:
# image dimensions
img.pixel_x_dimension, img.pixel_y_dimension

## Deploy

as WebApp

## Monitor

via Weights and Biases